In [1]:
import requests # used for getting data from a URL
from pprint import pprint # pretty print
import pandas as pd # used for turning results into mini databases
from solrq import Q # used to turn result queries into the right format
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda
import functools
import operator

search_url = "https://www.ebi.ac.uk/pdbe/search/pdb/select?" # the rest of the URL used for PDBe's search API.

In [ ]:
from MDAnalysis.analysis.base import AnalysisBase
from MDAnalysis.lib.distances import calc_dihedrals
class DihedralFromAtoms(AnalysisBase):
        """Calculate dihedral angles for specified atomgroups.
        Dihedral angles will be calculated for each atomgroup that is given for
        each step in the trajectory. Each :class:`~MDAnalysis.core.groups.AtomGroup`
        must contain 4 atoms.
        Note
        ----
        This class takes a list as an input and is most useful for a large
        selection of atomgroups. If there is only one atomgroup of interest, then
        it must be given as a list of one atomgroup.
        """

        def __init__(self, atomgroups, orders, **kwargs):
                """Parameters
                ----------
                atomgroups : list
                        a list of atomgroups for which the dihedral angles are calculated
                Raises
                ------
                ValueError
                        If any atomgroups do not contain 4 atoms
                """
                super(DihedralFromAtoms, self).__init__(
                        atomgroups[0].universe.trajectory, **kwargs)
                self.atomgroups = atomgroups

                if any([len(ag) != 4 for ag in atomgroups]):
                        raise ValueError("All AtomGroups must contain 4 atoms")

                if len(atomgroups) != len(orders):
                    raise ValueError("Order data should be provided for every atom group")

                self.ag1 = mda.AtomGroup([atomgroups[i][orders[i][0]] for i in range(len(atomgroups))])
                self.ag2 = mda.AtomGroup([atomgroups[i][orders[i][1]] for i in range(len(atomgroups))])
                self.ag3 = mda.AtomGroup([atomgroups[i][orders[i][2]] for i in range(len(atomgroups))])
                self.ag4 = mda.AtomGroup([atomgroups[i][orders[i][3]] for i in range(len(atomgroups))])


        def _prepare(self):
                self.angles = []

        def _single_frame(self):
                angle = calc_dihedrals(self.ag1.positions, self.ag2.positions,
                                                             self.ag3.positions, self.ag4.positions,
                                                             box=self.ag1.dimensions)
                self.angles.append(angle)

        def _conclude(self):
                self.angles = np.rad2deg(np.array(self.angles))

In [2]:
def make_request(search_dict, number_of_rows=10):
    """
    makes a get request to the PDBe API
    :param dict search_dict: the terms used to search
    :param number_of_rows: number or rows to return - limited to 10
    :return dict: response JSON
    """
    if 'rows' not in search_dict:
        search_dict['rows'] = number_of_rows
    search_dict['wt'] = 'json'
    #pprint(search_dict)
    response = requests.post(search_url, data=search_dict)

    if response.status_code == 200:
        return response.json()
    else:
        print("[No data retrieved - %s] %s" % (response.status_code, response.text))

    return {}

In [3]:
def format_search_terms(search_terms, filter_terms=None):
    ret = {'q': str(search_terms)}
    if filter_terms:
        fl = '{}'.format(','.join(filter_terms))
        ret['fl'] = fl
    return ret

In [4]:
def run_search(search_terms, filter_terms=None, number_of_rows=100):
    search_term = format_search_terms(search_terms, filter_terms)

    response = make_request(search_term, number_of_rows)
    results = response.get('response', {}).get('docs', [])
    print('Number of results: {}'.format(len(results)))
    return results

In [5]:
def run_search(search_terms, filter_terms=None, number_of_rows=100):
    search_term = format_search_terms(search_terms, filter_terms)

    response = make_request(search_term, number_of_rows)
    results = response.get('response', {}).get('docs', [])
    print('Number of results: {}'.format(len(results)))
    return results

In [6]:
def make_positive_angles(x):
    for i in range(len(x)):
        if x[i] < 0:
            x[i] = np.degrees(x[i]) + 360
        else:
            x[i] = np.degrees(x[i])
    return x

In [ ]:
def cmpAngles(dihL1, dihL2, maxAngDev):
    for i in range(len(dihL1)):
        if min((dihL1[i]-dihL2[i]) % 360,(dihL2[i]-dihL1[i]) % 360) > maxAngDev:
            return False
    return True

def parseByAngle(lipids,maxAngDev):
    out = []
    pdbl = []

    for i in range(len(lipids)):
        if checkZeros(lipids[i][6:],0.00001):
            continue
        if lipids[i][0] not in pdbl:
            out.append(lipids[i])
            pdbl.append(lipids[i][0])
        elif not angleQ(lipids[i],out,maxAngDev):
            out.append(lipids[i])
            pdbl.append(lipids[i][0])
    return out

def checkZeros(dihL, maxAngDev):
    for i in range(len(dihL)):
        if abs(dihL[i]) < maxAngDev:
            return True
    return False

def angleQ(dihL, l, maxAngDev):
    if len(l) == 0:
        return False
    for i in range(len(l)):
        if cmpAngles(dihL[6:],l[len(l)-1-i][6:],maxAngDev):
            return True
    return False

In [30]:
def calc_dihedrals_from_pdb(definitions,output,resolution):
    
    count = 0
    lipids = []
    it = 0
    ct = 0
    for RESnames in definitions:
        search_terms = Q(bound_compound_id = RESnames)
        #search_terms = Q(pdb_id = '6nhh')
        filter_terms = ['pdb_id', 'experimental_method','interacting_ligands','entity_id','resolution']
        first_results = run_search(search_terms, filter_terms)
        
        maxi = 2
        for it,i in enumerate(first_results):
            #if it > maxi:
            #    break
            print(ct,"analyzing: " + i.get('pdb_id') + " " + str(i.get('entity_id')))
            print(i.get('experimental_method'), str(i.get('resolution')))
            if "NMR" in str(i.get('experimental_method')):
                print("This is NMR structure")
                continue
            if i.get('resolution') > resolution:
                print("Resolution is too low")
                continue
            url = 'https://files.rcsb.org/download/' + i.get('pdb_id') + '.pdb'
            r = requests.get(url)
            pdbFILEpath = './' + i.get('pdb_id') + '.pdb' 
    
            with open(pdbFILEpath, 'wb') as f:
                f.write(r.content)
            
            try:
                t = mda.Universe(pdbFILEpath)
            except:
                print('pdb file' + i.get('pdb_id') + 'not found')
                continue
            
            resname = str(RESnames)
            names=functools.reduce(operator.iconcat,definitions[resname],[])
            for residue in t.select_atoms("name {} and resname {}".\
                                                 format(' '.join(set(names)),resname)).residues:
                lipid = [i.get('pdb_id'), i.get('entity_id'), i.get('experimental_method')[0], i.get('resolution'),\
                        residue.resid, residue.segid] \
                + [ 0.0 ] * len(definitions.get(RESnames))
                for dih in range(len(definitions.get(RESnames))):
                    # atom names for dihedral
                    atom1 = definitions.get(RESnames)[dih][0]
                    atom2 = definitions.get(RESnames)[dih][1]
                    atom3 = definitions.get(RESnames)[dih][2]
                    atom4 = definitions.get(RESnames)[dih][3]
                    # residue name
                    
                    # iterate by resid
                    sr = residue.atoms
                    atoms = sr.select_atoms\
                    ("name {} {} {} {}".format(atom1,atom2,atom3,atom4))
                    if atoms.n_atoms != 4:
                        print(atoms.n_atoms)
                        print("Atoms were not found from " + i.get('pdb_id'))
                        continue
                    ct += 1
                    order=[np.where(atoms.names==atom1)[0][0],
                            np.where(atoms.names==atom2)[0][0],
                            np.where(atoms.names==atom3)[0][0],
                            np.where(atoms.names==atom4)[0][0]]
                    R = DihedralFromAtoms([atoms],[order]).run()
                    lipid[6+dih] = R.angles.T[0,0]
                lipids = lipids + [lipid]
                #n1 = t.select_atoms("resname {} and name {}".format(str(RESnames),definitions.get(RESnames)[dih][0])).n_atoms
                #n2 = t.select_atoms("resname {} and name {}".format(str(RESnames),definitions.get(RESnames)[dih][1])).n_atoms
                #n3 = t.select_atoms("resname {} and name {}".format(str(RESnames),definitions.get(RESnames)[dih][2])).n_atoms
                #n4 = t.select_atoms("resname {} and name {}".format(str(RESnames),definitions.get(RESnames)[dih][3])).n_atoms
                #print(dih,n1,n2,n3,n4)
                
                #atom1=t.topology.select("resname == '" + str(RESnames) + "' and name == " + definitions.get(RESnames)[dih][0])
                #atom2=t.topology.select("resname == '" + str(RESnames) + "' and name == " + definitions.get(RESnames)[dih][1])
                #atom3=t.topology.select("resname == '" + str(RESnames) + "' and name == " + definitions.get(RESnames)[dih][2])
                #atom4=t.topology.select("resname == '" + str(RESnames) + "' and name == " + definitions.get(RESnames)[dih][3])
                #try:
                 #   index=[[atom1[0],atom2[0],atom3[0],atom4[0]]]
                #except:
                 #   print('Atoms were not found from ' + i.get('pdb_id'))
                  #  continue
                #print(index)
                #x=md.compute_dihedrals(t,index)
                #print(i.get('pdb_id'), make_positive_angles(x)[0],i.get('experimental_method'))
                #result = [make_positive_angles(x)[0][0] , i.get('pdb_id'), i.get('entity_id'), i.get('experimental_method')[0]]
                #print(result)
                #try:
                 #   if dihedrals[dih][-1][1] == result[1] and dihedrals[dih][-1][2] < result[2]:
                  #      dihedrals[dih][-1] = result
                   # else:
                    #    dihedrals[dih].append(result)                       
#                except:
 #                   dihedrals[dih].append(result)
                    
    return(lipids)
#    for dih in dihedrals:
#        dihVALUES = []
#        [dihVALUES.append(dihedrals[dih][i][0]) for i in range(len(dihedrals[dih])) if dihedrals[dih][i][0] not in dihVALUES]
#        fig= plt.figure(figsize=(5,5))
#        dist = plt.hist(dihVALUES, bins=range(0,360,20), density=True)
#        outfile=open('../Data/' + output + str(dih) + '.dat','w')
#        for i in range(len(dist[0])):
#            #print(str(dist[1][i]),str(dist[0][i]))
#            outfile.write(str(dist[1][i]) + " " + str(dist[0][i]) + "\n")
#        outfile.close()
#        outfileVALUES=open('../Data/' + output + str(dih) + 'values.dat','w')
#        for i in range(len(dihVALUES)):
#            outfileVALUES.write(str(dihVALUES[i]) + "\n")
#        outfileVALUES.close()
#        print("Number of data: ", len(dihVALUES))

In [46]:
def writelipids(lip,file):
    with open(file,"w") as f:
        f.write("{:<6}{:<7}{:<20}{:<10}{:<6}{:<6}{:<16}{:<16}{:<16}{:<16}{:<16}{:<16}\n".\
               format("pdbid","entity","method","res-ion","resid","chain","dih1","dih2","dih3","dih4","dih5","dih6"))
        for l in lip:
            f.write("{:<6}{:<7}{:<20}{:<10}{:<6}{:<6}{:<16.10f}{:<16.10f}{:<16.10f}{:<16.10f}{:<16.10f}{:<16.10f}\n".
                    format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10],l[11]))

In [72]:
res = 3.5
angCut = 3
PSnames = {"PSF" : 
           [["O1","CB","CA","N"],["P","O1","CB","CA"],["O2","P","O1","CB"],["C2","O2","P","O1"],["C3","C2","O2","P"],["C4","C3","C2","O2"]],           
          "PS6" :
           [["O1","CB","CA","N"],["P","O1","CB","CA"],["O2","P","O1","CB"],["C2","O2","P","O1"],["C3","C2","O2","P"],["C4","C3","C2","O2"]],
          "Q3G" :
           [["O28","C29","C30","N34"],["P25","O28","C29","C30"],["O24","P25","O28","C29"],["C23","O24","P25","O28"],["C22","C23","O24","P25"],["C21","C22","C23","O24"]],
          "P5S" :
           [["OG","CB","CA","N"],["P12","OG","CB","CA"],["O16","P12","OG","CB"],["C3","O16","P12","OG"],["C2","C3","O16","P12"],["C1","C2","C3","O16"]],
          "D39" :
           [["O","C1","C","N"],["P","O","C1","C"],["O3","P","O","C1"],["C2","O3","P","O"],["C3","C2","O3","P"],["C4","C3","C2","O3"]],
          "PS2" :
           [["O4P","C4","C5","N5"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
          "17F" :
          [["O3","C1","C2","N1"],["P1","O3","C1","C2"],["O6","P1","O3","C1"],["C4","O6","P1","O3"],["C5","C4","O6","P1"],["C6","C5","C4","O6"]],
          "8SP" :
          [["OG","CB","CA","N"],["P","OG","CB","CA"],["O3G","P","OG","CB"],["C3G","O3G","P","OG"],["C2G","C3G","O3G","P"],["C1G","C2G","C3G","O3G"]],
         }

PGnames = {"PGT" : 
           [["O4P","C4","C5","C6"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
           "PGK" : 
           [["O4P","C4","C5","C6"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
           "LHG" :
          [["O3","C3","C2","C1"],["P","O3","C3","C2"],["O6","P","O3","C3"],["C4","O6","P","O3"],["C5","C4","O6","P"],["C6","C5","C4","O6"]],
           "44G" : 
           [["O4P","C4","C5","C6"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C10","C2","C1","O3P"]],
           "PGV" : 
           [["O12","C04","C05","C06"],["P","O12","C04","C05"],["O11","P","O12","C04"],["C03","O11","P","O12"],["C02","C03","O11","P"],["C01","C02","C03","O11"]],
           "OZ2" : 
           [["OP3","CG1","CG2","CG3"],["P","OP3","CG1","CG2"],["O3G","P","OP3","CG1"],["C3G","O3G","P","OP3"],["C2G","C3G","O3G","P"],["C1G","C2G","C3G","O3G"]],
           "D3D" : 
           [["O5","C19","C20","C21"],["P","O5","C19","C20"],["O2","P","O5","C19"],["C18","O2","P","O5"],["C17","C18","O2","P"],["C16","C17","C18","O2"]],
         "PGW" : 
           [["O12","C04","C05","CAD"],["P","O12","C04","C05"],["O11","P","O12","C04"],["C03","O11","P","O12"],["C02","C03","O11","P"],["C01","C02","C03","O11"]],
         "DR9" : 
           [["OBN","CBO","CBP","CBQ"],["PBM","OBN","CBO","CBP"],["OBL","PBM","OBN","CBO"],["CBK","OBL","PBM","OBN"],["CBE","CBK","OBL","PBM"],["CAB","CBE","CBK","OBL"]],
         "P6L" : 
           [["O12","C1","C2","C50"],["P11","O12","C1","C2"],["O9","P11","O12","C1"],["C7","O9","P11","O12"],["C5","C7","O9","P11"],["C6","C5","C7","O9"]], 
          "PG8" :
          [["O3","C1","C2","C3"],["P1","O3","C1","C2"],["O6","P1","O3","C1"],["C4","O6","P1","O3"],["C5","C4","O6","P1"],["C6","C5","C4","O6"]],
          "H3T" :
          [["O26","C27","C28","C30"],["P23","O26","C27","C28"],["O22","P23","O26","C27"],["C21","O22","P23","O26"],["C01","C21","O22","P23"],["C02","C01","C21","O22"]],
           "GOT" :
          [["O26","C27","C28","C30"],["P23","O26","C27","C28"],["O22","P23","O26","C27"],["C21","O22","P23","O26"],["C01","C21","O22","P23"],["C02","C01","C21","O22"]],
          }

PCnames = {"PLC" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
          "PX4" :
          [["O3","C1","C2","N1"],["P1","O3","C1","C2"],["O4","P1","O3","C1"],["C6","O4","P1","O3"],["C7","C6","O4","P1"],["C8","C7","C6","O4"]],
          "6PL" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "LIO" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "HGX" : 
           [["O5","C4","C5","N1"],["P1","O5","C4","C5"],["O3P","P1","O5","C4"],["C1","O3P","P1","O5"],["C2","C1","O3P","P1"],["C3","C2","C1","O3P"]],
        "PC7" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "PC8" :
          [["O3","C1","C2","N1"],["P1","O3","C1","C2"],["O4","P1","O3","C1"],["C6","O4","P1","O3"],["C7","C6","O4","P1"],["C8","C7","C6","O4"]],
        "P1O" :
          [["O3","C1","C2","N1"],["P1","O3","C1","C2"],["O4","P1","O3","C1"],["C6","O4","P1","O3"],["C7","C6","O4","P1"],["C8","C7","C6","O4"]],
        "6O8" :
          [["O15","C16","C17","N18"],["P12","O15","C16","C17"],["O11","P12","O15","C16"],["C10","O11","P12","O15"],["C09","C10","O11","P12"],["C08","C09","C10","O11"]],
        "XP5" :
          [["O3","C1","C2","N1"],["P1","O3","C1","C2"],["O4","P1","O3","C1"],["C6","O4","P1","O3"],["CAM","C6","O4","P1"],["C8","CAM","C6","O4"]],
        "EGY" :
          [["O13","C11","C12","N"],["P","O13","C11","C12"],["O11","P","O13","C11"],["C1","O11","P","O13"],["C2","C1","O11","P"],["C3","C2","C1","O11"]],
        "PLD" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "SBM" : 
           [["O5","C4","C5","N1"],["P","O5","C4","C5"],["O3P","P","O5","C4"],["C1","O3P","P","O5"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "HXG" : 
           [["OAW","CAP","CAS","NBC"],["PBD","OAW","CAP","CAS"],["OAX","PBD","OAW","CAP"],["CAU","OAX","PBD","OAW"],["CBB","CAU","OAX","PBD"],["CAT","CBB","CAU","OAX"]],
        "PCW" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
          }

PEnames = {"8PE" :
          [["O13","C11","C12","N"],["P","O13","C11","C12"],["O11","P","O13","C11"],["C1","O11","P","O13"],
                ["C2","C1","O11","P"],["C3","C2","C1","O11"]],
        "PTY" :
          [["O11","C3","C2","N1"],["P1","O11","C3","C2"],["O14","P1","O11","C3"],["C5","O14","P1","O11"],
                ["C6","C5","O14","P1"],["C1","C6","C5","O14"]],
        "3PE" :
          [["O13","C11","C12","N"],["P","O13","C11","C12"],["O11","P","O13","C11"],["C1","O11","P","O13"],
                ["C2","C1","O11","P"],["C3","C2","C1","O11"]],
        "PEH" :
          [["O12","C11","C12","N"],["P","O13","C11","C12"],["O11","P","O13","C11"],["C1","O11","P","O13"],
                ["C2","C1","O11","P"],["C3","C2","C1","O11"]],
        "PEF" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],
                ["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "6OE" : 
           [["O04","C03","C02","N01"],["P05","O04","C03","C02"],["O08","P05","O04","C03"],
                ["C09","O08","P05","O04"],["C10","C09","O08","P05"],["C11","C10","C09","O08"]],
        "6O9" : 
           [["O04","C03","C02","N01"],["P05","O04","C03","C02"],["O08","P05","O04","C03"],
                ["C09","O08","P05","O04"],["C10","C09","O08","P05"],["C11","C10","C09","O08"]],
        "9PE" :
          [["O13","C11","C12","N"],["P","O13","C11","C12"],["O11","P","O13","C11"],["C1","O11","P","O13"],
                ["C2","C1","O11","P"],["C3","C2","C1","O11"]],
        "PEV" : 
           [["O4P","C4","C5","N6"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],
                ["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "46E" :
          [["O13","C11","C12","N"],["P","O13","C11","C12"],["O11","P","O13","C11"],["C1","O11","P","O13"],
                ["C2","C1","O11","P"],["C3","C2","C1","O11"]],
         "SBJ" : 
           [["O4P","C4","C5","N6"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],
                ["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "L9Q" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],
                ["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "PEK" : 
           [["O12","C04","C05","N"],["P","O12","C04","C05"],["O11","P","O12","C04"],["C03","O11","P","O12"],
                ["C02","C03","O11","P"],["C01","C02","C03","O11"]],
        "EPH" : 
           [["O8","C38","C39","N1"],["P1","O8","C38","C39"],["O5","P1","O8","C38"],["C37","O5","P1","O8"],
                ["C2","C37","O5","P1"],["C1","C2","C37","O5"]],
        "ZPE" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],
                ["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
        "9TL" : 
           [["OBA","CBB","CBC","NBD"],["PAZ","OBA","CBB","CBC"],["OAY","PAZ","OBA","CBB"],["CAX","OAY","PAZ","OBA"],
                ["CAW","CAX","OAY","PAZ"],["CAV","CAW","CAX","OAY"]],
        "9Y0" : 
           [["O1","C3","C4","N"],["P","O1","C3","C4"],["O3","P","O1","C3"],["C2","O3","P","O1"],
                ["C1","C2","O3","P"],["C","C1","C2","O3"]],
        "6OU" : 
           [["O26","C27","C28","N29"],["P23","O26","C27","C28"],["O22","P23","O26","C27"],["C21","O22","P23","O26"],
                ["C20","C21","O22","P23"],["C19","C20","C21","O22"]],
        "LOP" : 
           [["O1","C2","C1","N1"],["P1","O1","C2","C1"],["O2","P1","O1","C2"],["C3","O2","P1","O1"],
                ["C4","C3","O2","P1"],["C5","C4","C3","O2"]],
        "PEE" : 
           [["O4P","C4","C5","N"],["P","O4P","C4","C5"],["O3P","P","O4P","C4"],["C1","O3P","P","O4P"],
                ["C2","C1","O3P","P"],["C3","C2","C1","O3P"]],
                  }

lipids = calc_dihedrals_from_pdb(PSnames,'PS',res)
print(len(lipids))

writelipids(lipids,'./ps.csv')

lipidsparsed = parseByAngle(lipids,angCut)
writelipids(lipidsparsed,'./ps_parsed.csv')

lipids = calc_dihedrals_from_pdb(PGnames,'PG',res)
print(len(lipids))

writelipids(lipids,'./pg.csv')

lipidsparsed = parseByAngle(lipids,angCut)
writelipids(lipidsparsed,'./pg_parsed.csv')

lipids = calc_dihedrals_from_pdb(PCnames,'PC',res)
print(len(lipids))

writelipids(lipids,'./pc.csv')

lipidsparsed = parseByAngle(lipids,angCut)
writelipids(lipidsparsed,'./pc_parsed.csv')

lipids = calc_dihedrals_from_pdb(PEnames,'PE',res)
print(len(lipids))

writelipids(lipids,'./pe.csv')

lipidsparsed = parseByAngle(lipids,angCut)
writelipids(lipidsparsed,'./pe_parsed.csv')

Number of results: 34
0 analyzing: 3kaa 1
['X-ray diffraction'] 3.002
6 analyzing: 3bib 1
['X-ray diffraction'] 2.5
12 analyzing: 1dsy 1
['X-ray diffraction'] 2.6
18 analyzing: 6wlw 1
['Electron Microscopy'] 3.0
24 analyzing: 6wlw 2
['Electron Microscopy'] 3.0
30 analyzing: 6wlw 3
['Electron Microscopy'] 3.0
36 analyzing: 6wlw 4
['Electron Microscopy'] 3.0
42 analyzing: 6wlw 5
['Electron Microscopy'] 3.0
48 analyzing: 6wlw 6
['Electron Microscopy'] 3.0
54 analyzing: 6wlw 7
['Electron Microscopy'] 3.0
60 analyzing: 6wlw 8
['Electron Microscopy'] 3.0
66 analyzing: 6wm2 1
['Electron Microscopy'] 3.1
72 analyzing: 6wm2 2
['Electron Microscopy'] 3.1
78 analyzing: 6wm2 3
['Electron Microscopy'] 3.1
84 analyzing: 6wm2 4
['Electron Microscopy'] 3.1
90 analyzing: 6wm2 5
['Electron Microscopy'] 3.1
96 analyzing: 6wm2 6
['Electron Microscopy'] 3.1
102 analyzing: 6wm2 7
['Electron Microscopy'] 3.1
108 analyzing: 6wm2 8
['Electron Microscopy'] 3.1
114 analyzing: 6wm2 9
['Electron Microscopy'] 3.1
1

0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
1
Atoms were not found from 3rg1
2
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
1
Atoms were not found from 3rg1
2
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
1
Atoms were not found from 3rg1
2
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
0
Atoms were not found from 3rg1
1
Atoms were not found from 3rg1
2
Atoms were not found from 3rg1
12 analyzing: 4c7r 1
['X-ray diffraction'] 2.7
60 analyzing: 4doj 1
['X-ray diffraction'] 3.25
66 analyzing: 5azb 1
['X-ray diffraction'] 1.6
78 analyzing: 4jrb 1
['X-ray diffraction'] 2.414
1
Atoms were not 

/Users/pbuslaev/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: LI
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


0
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
1
Atoms were not found from 2cyd
2
Atoms were not found from 2cyd
3
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
1
Atoms were not found from 2cyd
2
Atoms were not found from 2cyd
3
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
1
Atoms were not found from 2cyd
2
Atoms were not found from 2cyd
3
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
1
Atoms were not found from 2cyd
2
Atoms were not found from 2cyd
3
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
1
Atoms were not found from 2cyd
2
Atoms were not found from 2cyd
3
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
0
Atoms were not found from 2cyd
1
Atoms were not found from 2cyd
2
Atoms were not found from 2cyd
3
Atoms were not found from 2cyd
0
Atoms we

/Users/pbuslaev/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: MN
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


...

7241 analyzing: 1kb9 1
['X-ray diffraction'] 2.3
7253 analyzing: 1kb9 2
['X-ray diffraction'] 2.3
7265 analyzing: 1kb9 3
['X-ray diffraction'] 2.3
7277 analyzing: 1kb9 4
['X-ray diffraction'] 2.3
7289 analyzing: 1kb9 5
['X-ray diffraction'] 2.3
7301 analyzing: 1kb9 6
['X-ray diffraction'] 2.3
7313 analyzing: 1kb9 7
['X-ray diffraction'] 2.3
7325 analyzing: 1kb9 8
['X-ray diffraction'] 2.3
7337 analyzing: 1kb9 9
['X-ray diffraction'] 2.3
7349 analyzing: 1kb9 10
['X-ray diffraction'] 2.3
7361 analyzing: 1kb9 11
['X-ray diffraction'] 2.3
7373 analyzing: 1eys 1
['X-ray diffraction'] 2.2
7379 analyzing: 1eys 2
['X-ray diffraction'] 2.2
7385 analyzing: 1eys 3
['X-ray diffraction'] 2.2
7391 analyzing: 1eys 4
['X-ray diffraction'] 2.2
7397 analyzing: 5klv 1
['X-ray diffraction'] 2.652
7403 analyzing: 5klv 2
['X-ray diffraction'] 2.652
7409 analyzing: 5klv 3
['X-ray diffraction'] 2.652
7415 analyzing: 5klv 4
['X-ray diffraction'] 2.652
7421 analyzing: 5klv 5
['X-ray diffraction'] 2.652
7427 ana

1
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
3
Atoms were not found from 5y5s
2
Atoms were not found from 5y5s
1
Atoms were not found from 5y5s
1
Atoms we

7790 analyzing: 6t15 3
['Electron Microscopy'] 3.29
7898 analyzing: 6t15 4
['Electron Microscopy'] 3.29
8006 analyzing: 6t15 5
['Electron Microscopy'] 3.29
8114 analyzing: 6t15 6
['Electron Microscopy'] 3.29
8222 analyzing: 6t15 7
['Electron Microscopy'] 3.29
8330 analyzing: 6t15 8
['Electron Microscopy'] 3.29
8438 analyzing: 6t15 9
['Electron Microscopy'] 3.29
8546 analyzing: 6t15 10
['Electron Microscopy'] 3.29
8654 analyzing: 6t15 11
['Electron Microscopy'] 3.29
8762 analyzing: 6t15 12
['Electron Microscopy'] 3.29
8870 analyzing: 6t15 13
['Electron Microscopy'] 3.29
8978 analyzing: 6t15 14
['Electron Microscopy'] 3.29
9086 analyzing: 6t15 15
['Electron Microscopy'] 3.29
9194 analyzing: 6t15 16
['Electron Microscopy'] 3.29
9302 analyzing: 6t15 17
['Electron Microscopy'] 3.29
9410 analyzing: 6t15 18
['Electron Microscopy'] 3.29
9518 analyzing: 6t15 19
['Electron Microscopy'] 3.29
9626 analyzing: 6t15 20
['Electron Microscopy'] 3.29
9734 analyzing: 6t15 21
['Electron Microscopy'] 3.29


11792 analyzing: 6rd5 1
['Electron Microscopy'] 2.69
11858 analyzing: 6rd5 2
['Electron Microscopy'] 2.69
11924 analyzing: 6rd5 3
['Electron Microscopy'] 2.69
11990 analyzing: 6rd5 4
['Electron Microscopy'] 2.69
12056 analyzing: 6rd5 5
['Electron Microscopy'] 2.69
12122 analyzing: 6rd5 6
['Electron Microscopy'] 2.69
12188 analyzing: 6rd5 7
['Electron Microscopy'] 2.69
12254 analyzing: 6rd5 8
['Electron Microscopy'] 2.69
12320 analyzing: 6xiw 2
['Electron Microscopy'] 2.8
12350 analyzing: 6xiw 1
['Electron Microscopy'] 2.8
12380 analyzing: 6lum 1
['Electron Microscopy'] 2.84
12416 analyzing: 6lum 2
['Electron Microscopy'] 2.84
12452 analyzing: 6lum 3
['Electron Microscopy'] 2.84
12488 analyzing: 6lum 4
['Electron Microscopy'] 2.84
12524 analyzing: 6lum 5
['Electron Microscopy'] 2.84
12560 analyzing: 6tyi 1
['Electron Microscopy'] 3.3
12578 analyzing: 6tyi 2
['Electron Microscopy'] 3.3
12596 analyzing: 7a24 1
['Electron Microscopy'] 3.8
Resolution is too low
12596 analyzing: 7a24 2
['Ele

KeyboardInterrupt: 